In [12]:
given 
feat.json(ctx给的feat.npy文件转化的) && ctx_joined_table_phone.csv(loanid - phone runlist) 
1. 我先通过 to_temp_file 函数，把符合窗口的 node_id 拿到
2. 跑query ctx_... 拿到符合条件的边 和 id2phone（损失一部分source 点，因为这些phone没有注册过（未知bug））-- edge.json 【id2phone.json】&&【edge 文件draft】
3. 通过符合条件的边 unique, 去找到 node_feat (损失一部分点， bug;没找出来原因; 这些phone 在node_feat.json 中不存在) 【feat 文件】
4. 删除edge.json 中 作为source node 缺不在 node_feat 字典中存在的元素；【edge 文件final】
5. 同上，重写【phone2id.json文件】
6. 扩充； 尽管大部分edge 的端点都是

6. 一个事实是， 尽管source node 全部都有 feat, 但是大量的 target nodes 是没feat vec 的；我的方案是0向量；但是不能读入内存，内存爆炸了，改相应的源码吧，不预先读了；；

'/home/qibo/all_project/Graph反欺诈/PYG'

In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv

dataset = 'Cora'

dataset = Planetoid('/home/qibo/all_project/Graph反欺诈/', dataset, T.TargetIndegree())
data = dataset[0]

data.train_mask = torch.zeros(data.num_nodes, dtype=torch.uint8)
data.train_mask[:data.num_nodes - 1000] = 1
data.val_mask = None
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.uint8)
data.test_mask[data.num_nodes - 500:] = 1

In [2]:
a = torch.randn(4, 4)
b = torch.max(a, 1)

In [5]:
b

torch.return_types.max(
values=tensor([1.0112, 0.9588, 0.5494, 1.4970]),
indices=tensor([1, 0, 0, 1]))

In [1]:
import json
import glob
import numpy as np
import scipy
import pandas as pd
from tqdm import tqdm
import time
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv, GCNConv
from torch_geometric.data import InMemoryDataset, download_url


In [2]:
save_root = '/data-0/tigergraph/30W_2/'
glob.glob(save_root+'/*')

['/data-0/tigergraph/30W_2/feat.json',
 '/data-0/tigergraph/30W_2/x_phone2idx.json',
 '/data-0/tigergraph/30W_2/id2phone_dic.json',
 '/data-0/tigergraph/30W_2/phone2idx.json',
 '/data-0/tigergraph/30W_2/label.json',
 '/data-0/tigergraph/30W_2/edge.json']

In [9]:
def read_cashbus_data(save_root):
    ##################### finish add test infos into the graph ############################
    with open(save_root+'/x_phone2idx.json') as json_file:
        x_phone2idx = json.load(json_file)
    with open(save_root+'/edge.json') as json_file:
        edge = json.load(json_file)
    with open(save_root+'/label.json') as json_file:
        label = json.load(json_file)
    with open(save_root+'/feat.json') as json_file:
        x = json.load(json_file)

    ##################### finish add test infos into the graph ############################
    feat_mat = []
    for k,v in tqdm(x.items()):
        feat_mat.append(v)
        
    labels = list(label.values())
    x = torch.tensor(feat_mat, dtype = torch.float)    
    y = torch.tensor(labels, dtype=torch.int64)
    
    edge = np.array(edge).T
    row1 = [x_phone2idx[str(i)] for i in edge[0]]
    row2 = [x_phone2idx[str(i)] for i in edge[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    
    ##################### finish all ############################
    data = Data(x=x, edge_index=new_edges, y=y)
    return data

class CashBus(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['feat.json', 'x_phone2idx.json', 'edge.json', 'label.json']

    @property
    def processed_file_names(self):
        return 'data12.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)

In [10]:
st = time.time()
root = '/home/qibo/all_project/Graph反欺诈/PYG/30W_data'
dataset = CashBus(root)
data = dataset[0]
print(time.time() - st)

processed_path:['/home/qibo/all_project/Graph反欺诈/PYG/30W_data/processed/data12.pt']
0.5365092754364014


In [11]:
data.train_mask = torch.zeros(len(data.y), dtype=torch.uint8)
data.train_mask[:data.num_nodes - 1000] = 1
data.val_mask = None
data.test_mask = torch.zeros(len(data.y), dtype=torch.uint8)
data.test_mask[data.num_nodes - 500:] = 1

In [20]:
torch.ones((data.edge_index.size(1),)).size()

torch.Size([3289057])

In [18]:
data.x.shape

torch.Size([14127, 14487])

In [ ]:
# return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

In [15]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True)
        self.conv1_1 = GCNConv(16, 16, cached=True)
        self.conv1_2 = GCNConv(16, 16, cached=True)
        self.conv2 = GCNConv(16, int(dataset.num_classes), cached=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        
        x = F.relu(self.conv1_1(x, edge_index))
        x = F.dropout(x, training=self.training)
        
        x = F.relu(self.conv1_2(x, edge_index))
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        print(data.y[mask].sum().item() / len(data.y[mask]))
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 2):
    print('epoch:{}'.format(epoch))
    st = time.time()
    train()
    print('use {} seconds'.format(time.time() - st))
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

epoch:1


IndexError: index 1200525 is out of bounds for dimension 0 with size 14127

In [5]:
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.uint8)
data.train_mask[:data.num_nodes - 1000] = 1
data.val_mask = None
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.uint8)
data.test_mask[data.num_nodes - 500:] = 1